In [109]:
import numpy as np  # Đại số tuyến tính (Numeric Python)
import pandas as pd  # Xử lý dữ liệu (Python Data Analysis)
from scipy import stats  # Thống kê (statistics), công cụ khoa học Python
from sklearn import preprocessing  # Tiền xử lý dữ liệu (Pre-processing)
from sklearn.feature_selection import SelectKBest, chi2  # Phân tích dữ liệu thăm dò

In [110]:
df = pd.read_csv('D:/PYPR (Python Programming)/G127CTNPNBAPlayer/G127CTNPNBAPLayer10EDA/NBA_2024_per_game(03-01-2024).csv')
print('Độ lớn của bảng [frame] dữ liệu cầu thủ:',df.shape) # Display the shape of the data set (Vẽ biểu đồ tập dữ liệu)
print(df[0:5]) # Display data (Hiển thị dữ liệu dạng mảng 5 dòng đầu)

Độ lớn của bảng [frame] dữ liệu cầu thủ: (538, 29)
             Player   Pos  Age   Tm   G  GS    MP   FG   FGA    FG%  ...  \
0  Precious Achiuwa  C-PF   24  TOT  26   0  17.2  3.0   6.6  0.453  ...   
1  Precious Achiuwa     C   24  TOR  25   0  17.5  3.1   6.8  0.459  ...   
2  Precious Achiuwa    PF   24  NYK   1   0   9.0  0.0   2.0  0.000  ...   
3       Bam Adebayo     C   26  MIA  23  23  34.1  8.0  15.7  0.506  ...   
4      Ochai Agbaji    SG   23  UTA  34  10  21.1  2.6   5.7  0.451  ...   

     FT%  ORB  DRB   TRB  AST  STL  BLK  TOV   PF   PTS  
0  0.571  2.0  3.4   5.3  1.7  0.6  0.5  1.1  1.6   7.4  
1  0.571  2.0  3.4   5.4  1.8  0.6  0.5  1.2  1.6   7.7  
2    NaN  1.0  2.0   3.0  0.0  0.0  0.0  0.0  2.0   0.0  
3  0.777  2.1  8.3  10.4  4.0  1.1  1.0  2.7  2.4  22.0  
4  0.714  0.8  1.7   2.5  1.0  0.6  0.5  0.7  1.5   6.6  

[5 rows x 29 columns]


In [111]:
print(df.count().sort_values()) 

FT%       476
3P%       489
2P%       521
FG%       530
eFG%      530
Pos       538
G         538
Player    538
FG        538
MP        538
GS        538
FGA       538
3PA       538
3P        538
Age       538
Tm        538
2PA       538
2P        538
FT        538
FTA       538
ORB       538
DRB       538
TRB       538
AST       538
STL       538
BLK       538
TOV       538
PF        538
PTS       538
dtype: int64


In [112]:
df = df.drop(columns=['FT%', '3P%', '2P%', 'FG%', 'eFG%'], axis=1)
print(df.shape) # kiểm tra lại số lượng cột & dòng của df sau khi XL NULL cột

(538, 24)


In [113]:
df = df.dropna(how='any')
print(df.shape) # kiểm tra lại số lượng cột & dòng của df sau khi XL NULL các dòng DL

(538, 24)


In [114]:
z = np.abs(stats.zscore(df._get_numeric_data())) # Dò tìm và lấy các giá trị cá biệt trong tập dữ liệu gốc thông qua điểm z (z_score)
print('MA TRAN Z-SCORE\n')
print(z) # in ra tập (ma trận) các giá trị z-score từ tập dữ liệu gốc

MA TRAN Z-SCORE

          Age         G        GS        MP        FG       FGA        3P  \
0    0.441876  0.570821  0.774672  0.125027  0.047157  0.016714  0.479979   
1    0.441876  0.476849  0.774672  0.096408  0.008423  0.021227  0.479979   
2    0.441876  1.778489  0.774672  0.907283  1.209167  0.889364  1.020886   
3    0.012239  0.288904  1.158337  1.487183  1.889527  1.709616  1.020886   
4    0.668933  1.322601  0.065767  0.247022  0.202092  0.187450  0.277290   
..        ...       ...       ...       ...       ...       ...       ...   
533  2.055757  1.214655  0.774672  1.346109  0.860564  1.003188  1.020886   
534  0.214818  0.946711  1.746644  1.725676  2.199396  2.563296  2.873641   
535  0.214818  0.006987  0.270409  0.649711  0.511961  0.604804  0.804523   
536  1.147527  0.288904  0.774672  0.850044  0.976765  0.946276  1.020886   
537  0.012239  1.134656  1.914732  0.800324  0.843717  0.286817  1.020886   

          3PA        2P       2PA  ...       FTA       ORB

In [115]:
df= df[(z < 54).all(axis=1)] #kiểm tra và chỉ giữ lại trong df các giá trị số liệu tưng ứng với z-score < 3  # {loại các giá trị >= 3} vì các giá trị z-score >=3 tướng ứng với số liệu quá khác biệt so với các số liệu còn lại (“cá biệt” = “ngoại lệ” = isolated}
print(df.shape) # xác định số dòng & cột dữ liệu sau khu xử lý các giá trị cá biệt

(538, 24)


In [116]:
# Mã hóa các giá trị trong cột 'Pos' bằng cách sử dụng 'replace'
df['Pos'].replace({'PG': 1, 'SG': 1, 'SF': 0, 'PF': 0, 'C': 0}, inplace=True)
# Xóa các dòng có giá trị 'Pos' không nằm trong danh sách đã định
df = df[df['Pos'].isin([1, 2, 3, 4, 5])]# Kiểm tra kết quả
# Kiểm tra lại phân phối các giá trị trong cột 'Pos'
print(df['Pos'])

4      1
6      1
7      1
9      1
13     1
      ..
516    1
520    1
525    1
532    1
534    1
Name: Pos, Length: 213, dtype: object


C:\Users\admin\AppData\Local\Temp\ipykernel_17468\3487941681.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Pos'].replace({'PG': 1, 'SG': 1, 'SF': 0, 'PF': 0, 'C': 0}, inplace=True)


In [117]:
df_num = df.drop(columns=['Player', 'Tm'], axis=1) 
rr = preprocessing.MinMaxScaler()  # xác định thang đo sẽ dùng để RR hóa theo khoảng đều 
rr.fit(df_num)  # Áp dụng thang đo vào data frame [df] của đề tài (đã tiền xử lý đến Bước 5)
df_num = pd.DataFrame(rr.transform(df_num), index=df.index, columns=df_num.columns)  # Chuyển đổi dữ liệu
df_num.iloc[4:10]  # Lấy 5 dòng dữ liệu từ chỉ số 4 đến 10
print(df_num)  # In ra DataFrame đã được chuẩn hóa


     Pos       Age         G        GS        MP        FG       FGA  \
4    0.0  0.210526  0.970588  0.294118  0.552198  0.228070  0.241525   
6    0.0  0.315789  0.911765  0.323529  0.593407  0.210526  0.245763   
7    0.0  0.473684  0.794118  0.823529  0.884615  0.394737  0.372881   
9    0.0  0.315789  0.529412  0.000000  0.417582  0.210526  0.241525   
13   0.0  0.210526  0.911765  0.000000  0.656593  0.421053  0.457627   
..   ...       ...       ...       ...       ...       ...       ...   
516  0.0  0.263158  0.000000  0.000000  0.164835  0.175439  0.127119   
520  0.0  0.263158  0.088235  0.000000  0.123626  0.131579  0.118644   
525  0.0  0.210526  0.676471  0.205882  0.546703  0.131579  0.182203   
532  0.0  0.631579  0.294118  0.000000  0.445055  0.131579  0.152542   
534  0.0  0.315789  0.852941  0.882353  0.978022  0.771930  0.855932   

           3P       3PA        2P  ...       FTA       ORB       DRB  \
4    0.255319  0.284483  0.137255  ...  0.047619  0.266667  0.2

In [118]:
X = df_num.loc[:, df_num.columns != 'Pos']  # Tất cả các cột trừ cột 'Pos'
y = df_num[['Pos']]  # Dữ liệu đầu ra là cột 'Pos'
# Loại bỏ các dòng có giá trị thiếu
selector = SelectKBest(chi2, k=3) # sd các hàm ... trong thư viện sklearn = Mô hình xác định các Thuộc tính quan trọng quyết định việc dự đoàn DL output = trích lọc Đặc trưng = Feature Extraction: k = 1...12 (đ/v bài này)
selector.fit(X, y) # Áp dụng mô hình trên vào ....
X_new = selector.transform(X) # Chuyên DL Input teho mô hình
print(X_new)
print(y)

[[0.14       0.3        0.19760479]
 [0.2        0.38       0.19760479]
 [0.28       0.44       0.39221557]
 [0.18       0.3        0.2005988 ]
 [0.36       0.54       0.42814371]
 [0.         0.02       0.        ]
 [0.1        0.         0.0748503 ]
 [0.78       0.7        0.73952096]
 [0.56       0.58       0.73652695]
 [0.1        0.18       0.08383234]
 [0.48       0.4        0.62874251]
 [0.38       0.38       0.54790419]
 [0.36       0.34       0.54491018]
 [1.         1.         0.56886228]
 [0.46       0.4        0.46706587]
 [0.12       0.4        0.35329341]
 [0.1        0.1        0.02994012]
 [0.14       0.34       0.1497006 ]
 [0.         0.         0.        ]
 [0.22       0.44       0.16467066]
 [0.2        0.5        0.5239521 ]
 [0.56       0.66       0.79041916]
 [0.04       0.04       0.13173653]
 [0.26       0.1        0.0508982 ]
 [0.26       0.28       0.2754491 ]
 [0.14       0.32       0.24850299]
 [0.28       0.26       0.48502994]
 [0.08       0.06       0.13

In [119]:
print('cac cot quan trong nhat quyet dinh cho Vector Output')
print(X.columns[selector.get_support(indices=True)]) # in ds các tt đặc trưng

cac cot quan trong nhat quyet dinh cho Vector Output
Index(['TOV', 'PF', 'PTS'], dtype='object')


In [120]:
df_new = df_num[['TOV','PF','PTS']] 
# TOV (Turnovers per game): Số lần cầu thủ mất bóng hoặc bị đối phương cướp bóng mỗi trận.
# PF (Personal fouls per game): Số lần cầu thủ phạm lỗi cá nhân mỗi trận.
# PTS (Points per game): Số điểm cầu thủ ghi được trung bình mỗi trận.
X = df_new[['PF']]
y = df_new[['PTS']]
print(X)
print(y)

       PF
4    0.30
6    0.38
7    0.44
9    0.30
13   0.54
..    ...
516  0.20
520  0.10
525  0.44
532  0.16
534  0.40

[213 rows x 1 columns]
          PTS
4    0.197605
6    0.197605
7    0.392216
9    0.200599
13   0.428144
..        ...
516  0.119760
520  0.098802
525  0.158683
532  0.146707
534  0.847305

[213 rows x 1 columns]
